In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import json as js
import os
from textwrap import wrap
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, confusion_matrix

#from matplotlib.ticker import MaxNLocator

In [ ]:
# creating/checking folder structure
paths = ["./dictionaries", "./embeddings","./results", "./tables", "./plots"]
for path in paths:
    if not os.path.isdir(path):
        os.mkdir(path)


### Loading results

In [ ]:
# loading the model specifications
with open("models.json", "r") as f:
    models = js.load(f)

# loading the gendered terms and names

with open("./dictionaries/gendered_terms_and_names.json", "r") as f:
    population_dict = js.load(f)

# loading stereotype dimensions
with open("./stereotype_dimensions.txt", "r") as f:
    stereotype_dimensions = [dimension[:-1] for dimension in f.readlines()]

stereotype_dictionary = pd.read_csv("./dictionaries/stereotype_dictionary_generated_examples.csv", index_col = 0)
additional_terms = pd.read_csv("./dictionaries/additional_terms_generated_examples.csv", index_col = 0)

In [ ]:
# loading the polar projections

stereotype_results = pd.read_csv("./results/stereotype_results_with_stats.csv", index_col = 0)

stereotype_results_additional_terms = pd.read_csv("./results/stereotype_results_additional_terms.csv", index_col = 0)

antonym_results = pd.read_csv("./results/antonym_results_with_stats.csv", index_col = 0)

stereotype_results.head()

In [ ]:
# loading the results from the validy experiment

accuracy_df = pd.read_csv("./results/additional_terms_accuracy.csv", index_col = 0)

# table for accuracy by term idx
term_idx_df = pd.read_csv("./results/additional_terms_accuracy_by_term_idx.csv", index_col = 0)
gen_term_idx_df = pd.read_csv("./results/additional_terms_accuracy_by_gen_term_idx.csv", index_col = 0)

In [ ]:
# setting line and marker styles for the plots

styles = {"line":
          {"Female Terms": "solid",
            "Female Names": "solid",
            "Male Terms": "dashdot",
            "Male Names": "dashdot"},
          "marker":
          {"Female Terms": "o",
            "Female Names": "d",
            "Male Terms": "o",
            "Male Names": "d"},
          "color":
          {"Female Terms": "forestgreen",
            "Female Names": "forestgreen",
            "Male Terms": "darkslateblue",
            "Male Names": "darkslateblue"}

         }


# setting the models for which to generate additional plots
selection = ["BERT-NoContext-L0",
             "BERT-DictExamples-L12",
             "BERT-GenExamples-L12",
            "BERT-SensePolar"]

# Properties of Generated Examples

In [ ]:
# plotting
fig, ax1 = plt.subplots(2, 2)

ax1[0, 0].hist(stereotype_dictionary["avg_example_length"])
ax1[0, 0].set_title("\n".join(wrap('n words - dictionary examples', 20)), fontweight="bold", size=20)

ax1[0,1].hist(stereotype_dictionary["average_example_term_idx"])
ax1[0, 1].set_title("\n".join(wrap('term position - dictionary examples', 20)), fontweight="bold", size=20)

ax1[1,0].hist(stereotype_dictionary["avg_gen_example_length"])
ax1[1, 0].set_title("\n".join(wrap('n words - generated examples', 20)), fontweight="bold", size=20)

ax1[1,1].hist(stereotype_dictionary["avg_gen_example_term_idx"])
ax1[1, 1].set_title("\n".join(wrap('term position - generated examples', 20)), fontweight="bold", size=20)

# adjust ax limits
for col in [0,1]:

    x_min = min(ax1[0, col].get_xlim()[0], ax1[1,col].get_xlim()[0])
    x_max = max(ax1[0, col].get_xlim()[1], ax1[1,col].get_xlim()[1])
    y_min = min(ax1[0, col].get_ylim()[0], ax1[1,col].get_ylim()[0])
    y_max = max(ax1[0, col].get_ylim()[1], ax1[1,col].get_ylim()[1])

    for row in [0,1]:
        ax1[row,col].set_xlim(x_min, x_max)
        ax1[row,col].set_ylim(y_min, y_max)
        ax1[row,col].set_xticks(range(0, int(x_max),5))
        ax1[row,col].set_xticklabels(range(0, int(x_max),5),fontsize = 17)
        ax1[row,col].set_yticks(range(0, int(y_max),40))
        ax1[row,col].set_yticklabels(range(0, int(y_max),40),fontsize = 17)



fig.set_figwidth(10)
fig.set_figheight(8)

fig.tight_layout()
plt.savefig(f'./plots/example_properties_for_dictionary_terms.pdf')

In [ ]:
# plotting
fig, ax1 = plt.subplots(2, 2)

# not counting examples twice for overlapping dimensions
additional_terms_ = additional_terms.loc[additional_terms["dimension"].isin(stereotype_dimensions)]

ax1[0, 0].hist(additional_terms_["avg_example_length"])
ax1[0, 0].set_title("\n".join(wrap('n words - dictionary examples', 20)), fontweight="bold", size=20)
ax1[0, 0].set_xlabel("n words")

ax1[0,1].hist(additional_terms_["average_example_term_idx"])
ax1[0, 1].set_title("\n".join(wrap('term position - dictionary examples', 20)), fontweight="bold", size=20)

ax1[1,0].hist(additional_terms_["avg_gen_example_length"])
ax1[1, 0].set_title("\n".join(wrap('n words - generated examples', 20)), fontweight="bold", size=20)

ax1[1,1].hist(additional_terms_["avg_gen_example_term_idx"])
ax1[1, 1].set_title("\n".join(wrap('term position - generated examples', 20)), fontweight="bold", size=20)

# adjust ax limits
for col in [0,1]:

    x_min = min(ax1[0, col].get_xlim()[0], ax1[1,col].get_xlim()[0])
    x_max = max(ax1[0, col].get_xlim()[1], ax1[1,col].get_xlim()[1])
    y_min = min(ax1[0, col].get_ylim()[0], ax1[1,col].get_ylim()[0])
    y_max = max(ax1[0, col].get_ylim()[1], ax1[1,col].get_ylim()[1])

    for row in [0,1]:
        ax1[row,col].set_xlim(x_min, x_max)
        ax1[row,col].set_ylim(y_min, y_max)
        ax1[row,col].set_xticks(range(0, int(x_max),5))
        ax1[row,col].set_xticklabels(range(0, int(x_max),5),fontsize = 17)
        ax1[row,col].set_yticks(range(0, int(y_max),200))
        ax1[row,col].set_yticklabels(range(0, int(y_max),200),fontsize = 17)



fig.set_figwidth(10)
fig.set_figheight(8)

fig.tight_layout()
plt.savefig(f'./plots/example_properties_for_additional_terms.pdf')

# Stereotype Dimensions across BERT Layers

In [ ]:
#overall accuracy plot

xticks = [i for i in range(13)]

for tokens in ["one_token_only", "all"]:

    selected_rows = accuracy_df.loc[accuracy_df["Tokens"] == tokens]

    fig, ax1 = plt.subplots(5)
    row = 0

    for model_group in ['BERT','AlBERT','GPT2','GPTNeo','Bloom']:

        layers = [i for i in range(0, 25, 2)] if model_group == "Bloom" else xticks

        model_rows = selected_rows.loc[selected_rows["Model"].str.startswith(model_group)]

        ax1[row].plot(xticks,model_rows.loc[model_rows["Model"].str.contains("DictExamples"),"Overall Accuracy"], label = "DictExamples", color ="teal", linestyle = "solid")
        ax1[row].plot(xticks,model_rows.loc[model_rows["Model"].str.contains("GenExamples"), "Overall Accuracy"],label = "GenExamples", color = "blue", linestyle = "dotted")
        ax1[row].plot(xticks,model_rows.loc[model_rows["Model"].str.contains("NoContext"), "Overall Accuracy"], label = "NoContext",color = "indigo", linestyle = "dashdot")

        ax1[row].set_title(model_group, size=15,style='italic')
        ax1[row].set_xticks(xticks)
        ax1[row].set_xticklabels(layers, size=12)
        ax1[row].set_ylim([0.35,0.85])
        ax1[row].set_yticks([0.4,0.5, 0.6, 0.7, 0.8])
        ax1[row].set_yticklabels([0.4,0.5, 0.6, 0.7, 0.8], size=12)
        row+=1

    ax1[0].legend(fontsize = 12)
    ax1[2].set_ylabel("Accuracy", size=15)
    ax1[4].set_xlabel("Layers", size=15)

    fig.set_figwidth(6)
    fig.set_figheight(10)
    fig.tight_layout()

    plt.savefig(f'./plots/stereotype_dimensions_across_layers_{tokens}.pdf')



## Accuracy for individual dimensions

In [ ]:

xticks = [i for i in range(13)]



selected_rows = accuracy_df.loc[accuracy_df["Tokens"] == "all"]

fig, ax1 = plt.subplots(5)
row = 0

for model_group in ['BERT-GenExamples','AlBERT-GenExamples','GPT2-GenExamples','GPTNeo-GenExamples','Bloom-GenExamples']:

    layers = [i for i in range(0, 25, 2)] if model_group == "Bloom-GenExamples" else xticks

    model_rows = selected_rows.loc[selected_rows["Model"].str.startswith(model_group)]


    for dimension in stereotype_dimensions:

        ax1[row].plot(xticks,model_rows[dimension], label = dimension)

    ax1[row].set_title(model_group, size=15, style='italic')
    ax1[row].set_xticks(xticks)
    ax1[row].set_xticklabels(layers, size=12)
    ax1[row].set_ylim([0.3,1])
    ax1[row].set_yticks([0.4, 0.6,  0.8,1])
    ax1[row].set_yticklabels([0.4, 0.6,  0.8,1], size=12)
    row+=1

ax1[0].legend(loc = "lower left", fontsize = 10)
ax1[2].set_ylabel("Accuracy", size=15)
ax1[4].set_xlabel("Layers", size=15)

fig.set_figwidth(6)
fig.set_figheight(10)
fig.tight_layout()

plt.savefig(f'./plots/individual_stereotype_dimensions_across_layers.pdf')


## Accuracy for Warmth and Competence

In [ ]:
xticks = [i for i in range(13)]

selected_rows = accuracy_df.loc[accuracy_df["Tokens"] == "all"]

fig, ax1 = plt.subplots(1,2)


col = 0

for dimension in ["Warmth", "Competence"]:

    for model_group in ['BERT-GenExamples','AlBERT-GenExamples','GPT2-GenExamples','GPTNeo-GenExamples','Bloom-GenExamples']:

        model_rows = selected_rows.loc[selected_rows["Model"].str.startswith(model_group)]

        ax1[col].plot(xticks,model_rows[dimension], label = model_group , linestyle = "solid" )


        layers = [i for i in range(0, 25, 2)] if model_group == "Bloom-GenExamples" else xticks

    ax1[col].set_title(dimension, size=15,style='italic')
    ax1[col].set_xticks(xticks)
    ax1[col].set_xticklabels(xticks, size=12)
    ax1[col].set_ylim([0.3,0.85])
    #ax1[row].set_yticks([0.4, 0.6,  0.8,1])
    #ax1[row].set_yticklabels([0.4, 0.6,  0.8,1], size=12)
    col+=1

ax1[0].legend(loc = "lower left", fontsize = 10)
ax1[0].set_ylabel("Accuracy", size=15)
ax1[0].set_xlabel("Layers$^\ddagger$", size=15)
ax1[1].set_xlabel("Layers$^\ddagger$", size=15)

fig.set_figwidth(12)
fig.set_figheight(4)
fig.tight_layout()

plt.savefig(f'./plots/warmth_competence_across_layers.pdf')


### Accuracy by Term Position

In [ ]:
# accuracy by term_idx plot

xticks = [i for i in range(7)]

fig, ax1 = plt.subplots(1,1)



for model in ['BERT-GenExamples','AlBERT-GenExamples','GPT2-GenExamples','GPTNeo-GenExamples','Bloom-GenExamples']:

    model_row = gen_term_idx_df.loc[gen_term_idx_df["Model"]== model, ["1","2","3","4","5","6","7+"]]

    ax1.plot(xticks,model_row.values[0], label = model , linestyle = "solid" )


        #layers = [i for i in range(0, 25, 2)] if model_group == "Bloom-GenExamples" else xticks

    #ax1.set_title(dimension, size=15,style='italic')
    ax1.set_xticks(xticks)
    ax1.set_xticklabels(["1","2","3","4","5","6","7+"], size=12)
    ax1.set_ylim([0.25,0.9])
    #ax1[row].set_yticks([0.4, 0.6,  0.8,1])
    #ax1[row].set_yticklabels([0.4, 0.6,  0.8,1], size=12)

ax1.legend(loc = "lower left", fontsize = 10)
ax1.set_ylabel("Accuracy", size=15)
ax1.set_xlabel("Term Position", size=15)
ax1.set_xlabel("Term Position", size=15)

fig.set_figwidth(8)
fig.set_figheight(4)
fig.tight_layout()

plt.savefig(f'./plots/accuracy_by_term_position_generated_examples.pdf')

In [ ]:
#warmth, competence accuracy by term_idx plot

xticks = [i for i in range(7)]

fig, ax1 = plt.subplots(1,1)



for model in ['BERT-DictExamples','AlBERT-DictExamples','GPT2-DictExamples','GPTNeo-DictExamples','Bloom-DictExamples']:

    model_row = term_idx_df.loc[term_idx_df["Model"]== model, ["1","2","3","4","5","6","7+"]]

    ax1.plot(xticks,model_row.values[0], label = model , linestyle = "solid" )


        #layers = [i for i in range(0, 25, 2)] if model_group == "Bloom-GenExamples" else xticks

    #ax1.set_title(dimension, size=15,style='italic')
    ax1.set_xticks(xticks)
    ax1.set_xticklabels(["1","2","3","4","5","6","7+"], size=12)
    ax1.set_ylim([0.25,0.9])
    #ax1[row].set_yticks([0.4, 0.6,  0.8,1])
    #ax1[row].set_yticklabels([0.4, 0.6,  0.8,1], size=12)

ax1.legend(loc = "lower left", fontsize = 10)
ax1.set_ylabel("Accuracy", size=15)
ax1.set_xlabel("Term Position", size=15)
ax1.set_xlabel("Term Position", size=15)

fig.set_figwidth(8)
fig.set_figheight(4)
fig.tight_layout()

plt.savefig(f'./plots/accuracy_by_term_position_dictionary_examples.pdf')

## Bias Profile of Stereotype Dimensions

In [ ]:
# selection and order of stereotype dimensions
plot_dimensions = ["Religion",
                   "Politics", "Status",
    "Agency", "Ability", "Competence", "Morality", "Sociability", "Warmth"]


# polar labels following the definition of the dimension by Nicolas
polar_labels = {"Warmth": {"low": "low WARMTH",
                          "high": "high WARMTH"},
                "Competence": {"low": "low COMPETENCE",
                          "high": "high COMPETENCE"},
    "Sociability": {"low": "low sociability",
                          "high": "high sociability"},
          "Morality": {"low": "low morality",
                          "high": "high morality"},
          "Ability": {"low": "low ability",
                     "high": "high ability"},
         "Agency": {"low": "low agency",
                     "high": "high agency"},
        "Status": {"low": "low status",
                     "high": "high status"},
        "Politics": {"low": "progressive",
                     "high": "traditional"},
        "Religion": {"low": "non-religious",
                     "high": "religious"}
         }



# looping over models
for model_group in ['BERT-GenExamples','AlBERT-GenExamples','GPT2-GenExamples','GPTNeo-GenExamples','Bloom-GenExamples']:

    layers = [i for i in range(0, 25, 2)] if model_group == "Bloom-GenExamples" else [i for i in range(13)]

    subplot_name1 = "Terms and Names Average Across Layers"
    subplot_name2 = "Stereotype Profile"


    cm = plt.get_cmap('viridis')
    cNorm  = colors.Normalize(vmin=layers[0], vmax=layers[-1])
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)

    # dictionary to collect the semantic differential labels
    plot_labels = {subplot_name1:{
                "low labels": [],
                 "high labels": []},
                   subplot_name2:{
                "low labels": [],
                 "high labels": []}
                  }


    # dictionary to collect the values to plot
    plot_values = {subplot_name1: {layer:[] for layer in layers},
                   subplot_name2: {"Female Terms": [],
                            "Female Names": [],
                            "Male Terms": [],
                            "Male Names": []}
                  }


    # filling the dictionaries
    for dimension in plot_dimensions:

        for layer in layers:

            row = stereotype_results.loc[(stereotype_results["Model"] == model_group + "-L" + str(layer)) & (stereotype_results["Dimension"] == dimension)]

            #plot_values["Female Terms"][layer].append(row["female_terms_mean"])
            mean = (row["female_terms_mean"]+row["male_terms_mean"]+row["female_names_mean"]+ row["male_names_mean"])/4
            plot_values[subplot_name1][layer].append(mean)
            #plot_values["Female Names"].append(row["female_names_mean"])
            #plot_values["Female Bias"][layer].append(row["female_terms_mean"]-mean)
            #plot_values["Male Bias"][layer].append(row["male_terms_mean"]-mean)
            #plot_values["Male Names"].append(row["male_names_mean"])

        row = stereotype_results.loc[(stereotype_results["Model"] == model_group) & (stereotype_results["Dimension"] == dimension)]

        mean = (row["female_terms_mean"]+row["male_terms_mean"]+row["female_names_mean"]+ row["male_names_mean"])/4

        plot_values[subplot_name2]["Female Terms"].append((row["female_terms_mean"]-mean))
        plot_values[subplot_name2]["Female Names"].append((row["female_names_mean"]-mean))
        plot_values[subplot_name2]["Male Terms"].append((row["male_terms_mean"]-mean))
        plot_values[subplot_name2]["Male Names"].append((row["male_names_mean"]-mean))

        # set polar labels, with significance
        low_label = polar_labels[dimension]["low"]
        high_label = polar_labels[dimension]["high"]

        plot_labels[subplot_name1]["low labels"].append(low_label)
        plot_labels[subplot_name1]["high labels"].append(high_label)

        if row["gendered_terms_diff_pvalue"].values[0] <0.05:
            high_label = high_label + "$^*$"

        if row["names_diff_pvalue"].values[0] < 0.05:
             high_label = high_label + "$^\ddagger$"

        plot_labels[subplot_name2]["low labels"].append(low_label)
        plot_labels[subplot_name2]["high labels"].append(high_label)


    # plotting
    fig, ax1 = plt.subplots(1, 2)
    col = 0

    for subplot, lines in plot_values.items():


        #if words == "All Terms":
        for line, values in lines.items():
            if subplot == subplot_name1:
                color = scalarMap.to_rgba(line)
                marker = None
            else:
                color = styles["color"][line]
                marker = styles["marker"][line]
            ax1[col].plot(values, np.arange(len(values)), label = str(line), color = color, marker = marker)#,  color = styles["color"][words], linestyle = styles["line"][words])

        # centering the plot
        max_abs_value = abs(max([j for i in [j for i in plot_values[subplot].values() for j in i] for j in i],key=abs))
        ax1[col].set_xlim(-max_abs_value-0.1*max_abs_value, max_abs_value+0.1*max_abs_value)

        #ax1[col].hlines(y=6.5, xmin=-max_abs_value-0.5*max_abs_value, xmax=max_abs_value+0.5*max_abs_value, linewidth=1, linestyle = "dotted",color='black')

        # setting left side tick labels
        ax1_tick_labels = plot_labels[subplot]["low labels"]
        ax1[col].set_yticks(np.arange(len(ax1_tick_labels)))
        ax1[col].set_yticklabels(ax1_tick_labels)
        #ax1.xaxis.set_major_locator(MaxNLocator(integer=True)) # depending on the range of values

        # setting right side tick labels
        ax2_tick_labels = plot_labels[subplot]["high labels"]
        ax2 = ax1[col].twinx()
        ax2.set_ylim(ax1[col].get_ylim())
        ax2.set_yticks(np.arange(len(ax2_tick_labels)))
        ax2.set_yticklabels(ax2_tick_labels)



    # possibly at line at x =0

        ax1[col].legend(loc="lower left")
        ax1[col].set_title(subplot)

        col+=1

    fig.suptitle(model_group)
    fig.set_figwidth(12)
    fig.set_figheight(4)
    fig.tight_layout()
    plt.savefig(f'./plots/stereotype_profile_{model_group}.pdf')



### last layers

In [ ]:
# selection and order of stereotype dimensions
plot_dimensions = ["Religion",
                   "Politics", "Status",
    "Agency", "Ability", "Competence", "Morality", "Sociability", "Warmth"]


# polar labels following the definition of the dimension by Nicolas
polar_labels = {"Warmth": {"low": "low WARMTH",
                          "high": "high WARMTH"},
                "Competence": {"low": "low COMPETENCE",
                          "high": "high COMPETENCE"},
    "Sociability": {"low": "low sociability",
                          "high": "high sociability"},
          "Morality": {"low": "low morality",
                          "high": "high morality"},
          "Ability": {"low": "low ability",
                     "high": "high ability"},
         "Agency": {"low": "low agency",
                     "high": "high agency"},
        "Status": {"low": "low status",
                     "high": "high status"},
        "Politics": {"low": "progressive",
                     "high": "traditional"},
        "Religion": {"low": "non-religious",
                     "high": "religious"}
         }



# looping over models
for model in ['BERT-GenExamples-L12','AlBERT-GenExamples-L12','GPT2-GenExamples-L12','GPTNeo-GenExamples-L12','Bloom-GenExamples-L12']:

    # dictionary to collect the semantic differential labels
    plot_labels = {"low labels": [],
                 "high labels": []}

    # dictionary to collect the values to plot
    plot_values = {"Female Terms": [],
                "Female Names": [],
                "Male Terms": [],
            "Male Names": []
                  }

    # filling the dictionaries
    for dimension in plot_dimensions:

        row = stereotype_results.loc[(stereotype_results["Model"] == model) & (stereotype_results["Dimension"] == dimension)]

        if len(row)== 1:

            plot_values["Female Terms"].append(row["female_terms_mean"])
            plot_values["Female Names"].append(row["female_names_mean"])
            plot_values["Male Terms"].append(row["male_terms_mean"])
            plot_values["Male Names"].append(row["male_names_mean"])


            # set polar labels, with significance
            low_label = polar_labels[dimension]["low"]
            high_label = polar_labels[dimension]["high"]

            if row["gendered_terms_diff_pvalue"].values[0] <0.05:
                high_label = high_label + "$^*$"

            if row["names_diff_pvalue"].values[0] < 0.05:
                high_label = high_label + "$^\ddagger$"

            plot_labels["low labels"].append(low_label)
            plot_labels["high labels"].append(high_label)


    # plotting
    fig, ax1 = plt.subplots(1, 1)

    for words, values in plot_values.items():
        ax1.plot(values, np.arange(len(values)), label = words, marker = styles["marker"][words],  color = styles["color"][words], linestyle = styles["line"][words])

    # centering the plot
    max_abs_value = abs(max([j for i in [j for i in plot_values.values() for j in i] for j in i],key=abs))
    ax1.set_xlim(-max_abs_value-0.1*max_abs_value, max_abs_value+0.1*max_abs_value)


    # setting left side tick labels
    ax1_tick_labels = plot_labels["low labels"]
    ax1.set_yticks(np.arange(len(ax1_tick_labels)))
    ax1.set_yticklabels(ax1_tick_labels)
    #ax1.xaxis.set_major_locator(MaxNLocator(integer=True)) # depending on the range of values

    # setting right side tick labels
    ax2_tick_labels = plot_labels["high labels"]
    ax2 = ax1.twinx()
    ax2.set_ylim(ax1.get_ylim())
    ax2.set_yticks(np.arange(len(ax2_tick_labels)))
    ax2.set_yticklabels(ax2_tick_labels)

    # possibly at line at x =0

    ax1.legend(loc="lower left")
    ax1.set_title(model)

    fig.set_figwidth(6)
    fig.set_figheight(4)
    fig.tight_layout()
    plt.savefig(f'./plots/stereotype_profile_{model}.pdf')



### Top 10 Most Biased Antonym Pairs

In [ ]:

# selecting the criteria for sorting the antonym pairs
criteria = "gendered_terms_diff_abs"


# looping over models
for model in ["BERT-DictExamples-L12"]:

    # filtering the dataframe
    plot_df = antonym_results.loc[(antonym_results["Model"] == model) ].sort_values(criteria, ascending = False).head(10)


    # dictionary to collect the semantic differential labels
    plot_labels = {"low labels": [],
                 "high labels": []}

    # dictionary to collect the values to plot
    plot_values = {"Female Terms": [],
                "Female Names": [],
                "Male Terms": [],
                "Male Names": []}

    # filling the dictionaries
    for index, row in plot_df.iloc[::-1].iterrows():

        plot_values["Female Terms"].append(row["female_terms_mean"])
        plot_values["Female Names"].append(row["female_names_mean"])
        plot_values["Male Terms"].append(row["male_terms_mean"])
        plot_values["Male Names"].append(row["male_names_mean"])


        # set polar labels, with significance
        low_label = row["term1"]
        high_label = row["term2"]

        if row["gendered_terms_diff_pvalue"] <0.05:
            high_label = high_label + "$^*$"

        if row["names_diff_pvalue"] < 0.05:
            high_label = high_label + "$^\ddagger$"

        plot_labels["low labels"].append(low_label)
        plot_labels["high labels"].append(high_label)




    # plotting
    fig, ax1 = plt.subplots(1, 1)

    for words, values in plot_values.items():
        ax1.plot(values, np.arange(len(values)), label = words, marker = styles["marker"][words],  color = styles["color"][words], linestyle = styles["line"][words])


    # centering the plot
    max_abs_value = abs(max([j for i in plot_values.values() for j in i],key=abs))
    ax1.set_xlim(-max_abs_value-0.1*max_abs_value, max_abs_value+0.1*max_abs_value)

    # setting left side tick labels
    ax1_tick_labels = plot_labels["low labels"]
    ax1.set_yticks(np.arange(len(ax1_tick_labels)))
    ax1.set_yticklabels(ax1_tick_labels)
    #ax1.xaxis.set_major_locator(MaxNLocator(integer=True)) # depending on the range of values

    # setting right side tick labels
    ax2_tick_labels = plot_labels["high labels"]
    ax2 = ax1.twinx()
    ax2.set_ylim(ax1.get_ylim())
    ax2.set_yticks(np.arange(len(ax2_tick_labels)))
    ax2.set_yticklabels(ax2_tick_labels)

    ax1.legend()


    fig.tight_layout()
    plt.savefig(f'./plots/top10_biased_antonym_pairs_{model}.pdf')

# Cover Plot Handpicked Example

In [ ]:
label_dict = {"female_names_mean": "Female Names",
             "female_terms_mean": "Female Terms",
              "male_names_mean": "Male Names",
             "male_terms_mean": "Male Terms"
             }


for model_group in ['BERT-GenExamples','AlBERT-GenExamples','GPT2-GenExamples','GPTNeo-GenExamples','Bloom-GenExamples']:

    model_rows = stereotype_results.loc[stereotype_results["Model"] == model_group]

    warmth = model_rows.loc[model_rows["Dimension"] == "Warmth"]
    competence = model_rows.loc[model_rows["Dimension"] == "Competence"]

    fig, ax1 = plt.subplots(1, 1)


    female_population = ["female_terms_mean", "female_names_mean"]
    male_population = ["male_terms_mean", "male_names_mean"]
    population = female_population + male_population

    marker = {word:"s" for word in female_population}
    marker.update({word:"d" for word in male_population})

    color = {word: "forestgreen" for word in female_population}
    color.update({word: "darkslateblue" for word in male_population})

    xlim1 = min([warmth[term].values[0] for term in population])
    xlim2 = max([warmth[term].values[0] for term in population])
    ax1.set_xlim(xlim1-0.1*(abs(xlim2-xlim1)), xlim2+0.35*(xlim2-xlim1))

    ylim1 = min([competence[term].values[0] for term in population])
    ylim2 = max([competence[term].values[0] for term in population])
    ax1.set_ylim(ylim1-0.2*(abs(ylim2-ylim1)), ylim2+0.1*(ylim2-ylim1))

    ax1.set_xlabel("Warmth", size=15)
    ax1.set_ylabel("Competence", size=15)

    ax1.set_title(model_group, fontstyle = "italic", size=15)


    for word in population:

        x  = warmth[word].values[0]
        y = competence[word].values[0]

        ax1.plot(x, y, marker[word], label = word, color = color[word])

        if word in ["female_names_mean", "female_terms_mean"]:

            ax1.text(x+0.02*abs((xlim2-xlim1)),y, label_dict[word],size=15)
        else:
            ax1.text(x+0.02*abs((xlim2-xlim1)),y-0.125*abs((ylim2-ylim1)), label_dict[word],size=15)


    fig.set_figwidth(6)
    fig.set_figheight(2.5)
    fig.tight_layout()
    plt.savefig(f'./plots/cover_plot_{model_group}.pdf')



